# Vault and account transfers



This notebook aims to provide a brief demonstration of how this codebase could be used for visualize vault and account transfers.

In [2]:
"""load dependencies and define constants"""
import sys
from enum import Enum
from decimal import Decimal

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv

sys.path.append("..")
from src.yearn import Network, Yearn, Subgraph
from src.utils.web3 import Web3Provider 

# constants
FROM_BLOCK = 13900000  # Dec-29-2021
TOP_COUNT = 10

class Stats(Enum):
    # top-level stats
    vaults = "vaults"
    overall_account_transfers = "overall_account_transfers"
    # sub stats
    count = "count"
    shares = "shares"
    shares_usdc = "shares_usdc"
    account_transfers = "account_transfers"

    def __repr__(self):
        return self.value

# Used to compare performances
metrics = [Stats.count, Stats.shares, Stats.shares_usdc]

pd.set_option("expand_frame_repr", False)

In [3]:
"""get current block, load vaults from the yearn instance and initialize subgraph instance"""
load_dotenv()
w3 = Web3Provider(Network.Mainnet)
current_block = w3.provider.eth.get_block("latest")["number"]
print(f"current block number: {current_block}")

yearn = Yearn(Network.Mainnet)
vaults = yearn.vaults
print(f"loaded metadata for {len(vaults)} vaults (v2)")

subgraph = Subgraph(Network.Mainnet)

current block number: 14976389
loaded metadata for 101 vaults (v2)


In [4]:
"""helper function to convert dict to reversed sorted dataframes"""
def to_reversed_sorted_df(_dict: dict[str, dict]):
    stats = []
    for address, sub_dict in _dict.items():
        # only include address and items in metrics
        new_sub_dict = dict((k, v) for k, v in sub_dict.items() if k in metrics)
        stats.append((address, *new_sub_dict.values()))
    df = pd.DataFrame(stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.columns.name = df.index.name
    df.index.name = None
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    return df

In [5]:
"""get vaults, accounts and overall accounts' stats for withdrawal"""
"""print stats for each vault and top accounts by shares_usdc for each vault"""
withdrawals_stats = {Stats.vaults: {}, Stats.overall_account_transfers: {}}
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue

    transfers = subgraph.transfers("withdrawals", vault, from_block=FROM_BLOCK, to_block=current_block)

    # current vault's stats
    _dict = {
        Stats.count: transfers.count,
        Stats.shares: transfers.shares,
        Stats.shares_usdc: float(Decimal(transfers.shares) * share_price),
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in _dict)}")

    account_items = transfers.account_transfers.items()
    sorted_account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    account_transfers = {}
    
    # loop current vault's account_transfers
    for index, (account, transfer) in enumerate(sorted_account_transfers):
        # current vault's single account_transfers stat
        sub_dict = {
            Stats.count: transfer.count,
            Stats.shares: transfer.shares,
            Stats.shares_usdc: float(Decimal(transfer.shares) * share_price)
        }
        account_transfers[account] = sub_dict

        # print top accounts' stats for current vault
        if index < TOP_COUNT:
            print(f"{index}: {account} {sub_dict}")

        # add overall_account_transfers to withdrawals stats
        overall_account_transfers = withdrawals_stats[Stats.overall_account_transfers]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + sub_dict[k] for k in sub_dict}

    # add vault's stats to withdrawals stats
    withdrawals_stats[Stats.vaults][vault.address] = {**_dict, Stats.account_transfers: account_transfers}
    print()

vault: 0x790a60024bC3aea28385b60480f15a0771f26D09 {count: 9, shares: 161.40082078679598, shares_usdc: 721503.297739477}
0: 0x4ac68d9a53f699439dc60ddcab304a9d138d7946 {count: 1, shares: 133.84776873416791, shares_usdc: 598334.0485259381}
1: 0x7338afb07db145220849b04a45243956f20b14d9 {count: 1, shares: 20.20415112596147, shares_usdc: 90317.76662811448}
2: 0xc4b74ae89ef93e683fb4afba8e118d730313d45a {count: 2, shares: 5.137756217892914, shares_usdc: 22967.095434340557}
3: 0x822f791de0c49f7b73810bf718772b13a69fa9e1 {count: 1, shares: 0.6165173260973426, shares_usdc: 2755.991461036903}
4: 0x1db67d560813ea7aba48bd8a9429cbecbeb2118e {count: 1, shares: 0.4922562165003181, shares_usdc: 2200.512251464294}
5: 0xd6e3e771a271c7d773583f9770fc112af15a916f {count: 1, shares: 0.44057635491088615, shares_usdc: 1969.4899407862724}
6: 0xd19a40c9ad255ac0ea9f771f0d6f08d0a84f1554 {count: 1, shares: 0.35213874859043787, shares_usdc: 1574.1510305295712}
7: 0xf46ff05e1a1616d314784de85554c7417abdce82 {count: 1, s

2022-06-17 09:08:10.222 WARNING connectionpool: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.etherscan.io', port=443): Read timed out. (read timeout=10)")': /api?&apiKey=5RW3TIB9G6X4Y9NP136F1PHJNFP76XNH63&address=0x83d95e0D5f402511dB06817Aff3f9eA88224B030&module=contract&action=getabi
2022-06-17 09:08:25.047 WARNING connectionpool: Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.etherscan.io', port=443): Read timed out. (read timeout=10)")': /api?&apiKey=5RW3TIB9G6X4Y9NP136F1PHJNFP76XNH63&address=0x83d95e0D5f402511dB06817Aff3f9eA88224B030&module=contract&action=getabi
2022-06-17 09:08:43.875 WARNING connectionpool: Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.ether

vault: 0x8414Db07a7F743dEbaFb402070AB01a4E0d2E45e {count: 21, shares: 19.092704244730704, shares_usdc: 394599.28399770585}
0: 0xd6b88257e91e4e4d4e990b3a858c849ef2dfde8c {count: 7, shares: 8.355533438916394, shares_usdc: 172688.3457761225}
1: 0x8792a8cda2828b1cdfd7a349eca40d25399256f9 {count: 1, shares: 3.6461650808209143, shares_usdc: 75357.27321740972}
2: 0xba90e5b6d3376d792ca3927524c27a185fbfb159 {count: 1, shares: 1.5018778935625647, shares_usdc: 31040.125791260307}
3: 0xc47fae56f3702737b69ed615950c01217ec5c7c8 {count: 1, shares: 1.0374888506321407, shares_usdc: 21442.34532559899}
4: 0xb4367abe9d87c508eceb60c422cbbf8e34aa8dc9 {count: 1, shares: 1.0294959049301264, shares_usdc: 21277.150777429222}
5: 0xce34448192a400305b948c3719bd9231806e8c0b {count: 1, shares: 1.012644067123143, shares_usdc: 20928.864696660105}
6: 0x7ad456d9efced426fe6a9c6f70e32ac83438ae8e {count: 1, shares: 0.9984927996511889, shares_usdc: 20636.39276913657}
7: 0xc842d55c3a0371e060ff00bd66300ba097d295e3 {count: 1, 

In [6]:
"""top withdrawals for vaults"""
df = to_reversed_sorted_df(withdrawals_stats[Stats.vaults])
print(f"top withdrawals for {Stats.vaults} by approx. USDC value")
df.head(TOP_COUNT)


top withdrawals for Stats.vaults by approx. USDC value


address,count,shares,approx. USDC value
0xdA816459F1AB5631232FE5e97a05BBBb94970c95,1732,5.500483e+08,5.503811e+08
0x27b7b1ad7288079A66d12350c828D3C00A6F07d7,353,3.304172e+08,3.439574e+08
0xa258C4606Ca8206D8aA700cE2143D7db854D168c,1818,2.923662e+05,3.125081e+08
0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE,907,2.228766e+08,2.223645e+08
0xdCD90C7f6324cfa40d7169ef80b12031770B4325,1302,1.981180e+05,2.208232e+08
0xB4AdA607B9d6b2c9Ee07A275e9616B84AC560139,166,1.462479e+08,1.475268e+08
0x5f18C75AbDAe578b483E5F43f12a39cF75b973a9,959,1.148020e+08,1.146987e+08
0x7Da96a3891Add058AdA2E826306D812C638D87a7,570,8.935392e+07,8.913080e+07
0x3B96d491f067912D18563d56858Ba7d6EC67a6fa,483,8.144328e+07,8.658423e+07
0x2DfB14E32e2F8156ec15a2c21c3A6c053af52Be8,99,7.841150e+07,7.898304e+07


In [7]:
"""top withdrawals for overall_account_transfers"""
df = to_reversed_sorted_df(withdrawals_stats[Stats.overall_account_transfers])
print(f"top withdrawals for {Stats.overall_account_transfers.value.replace('_', ' ')} by approx. USDC value")
df.head(TOP_COUNT)


top withdrawals for overall account transfers by approx. USDC value


address,count,shares,approx. USDC value
0x6fe02be0ec79dcf582cbdb936d7037d2eb17f661,87,2.881235e+08,2.882978e+08
0xd6b88257e91e4e4d4e990b3a858c849ef2dfde8c,1838,5.161973e+07,2.200744e+08
0xb039ea6153c827e59b620bdcd974f7bbfe68214a,962,1.823751e+08,1.824855e+08
0x6d75657771256c7a8cb4d475fdf5047b70160132,154,1.002245e+05,1.071292e+08
0xdb91f52eefe537e5256b8043e5f7c7f44d81f5aa,12,1.303257e+10,1.035555e+08
0xa6d1c610b3000f143c18c75d84baa0ec22681185,28,9.830915e+07,1.023378e+08
0x0c8f62939aeee6376f5fac88f48a5a3f2cf5debb,34,9.472795e+07,9.860981e+07
0x0d5dc686d0a2abbfdafdfb4d0533e886517d4e83,1,5.629800e+07,5.670836e+07
0x65a8efc842d2ba536d3f781f504a1940f61124b4,13,5.118207e+07,5.327946e+07
0xf9fdc2b5f60355a237deb8bd62cc117b1c907f7b,20,4.640823e+04,5.172682e+07


In [ ]:
# TODO: WIP
# """distribution of the top 10 wallets"""
# net_boost = df_boost.copy()
# net_boost[net_boost < 0] = 0.0

# sns.set_style('whitegrid')
# plt.figure(figsize=(12, 8))
# plt.tick_params(labelbottom=False)
# plt.bar(net_boost.index[:10], net_boost.value.iloc[:10] / net_boost.value.sum())
# plt.title("Net Deposit of the Top 10 Wallets in yvBOOST (in %)")

# concentration = net_boost.value.iloc[:10].sum() / net_boost.value.sum()
# print("Concentration: {:.2f}%".format(concentration * 100))

In [9]:
"""get vaults, accounts and overall accounts' stats for deposits"""
"""print stats for each vault and top accounts by shares_usdc for each vault"""
deposits_stats = {Stats.vaults: {}, Stats.overall_account_transfers: {}}
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue
    
    transfers = subgraph.transfers("deposits", vault, from_block=FROM_BLOCK, to_block=current_block)

    # current vault's stats
    _dict = {
        Stats.count: transfers.count,
        Stats.shares: transfers.shares,
        Stats.shares_usdc: float(Decimal(transfers.shares) * share_price)
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in _dict)}")

    account_items = transfers.account_transfers.items()
    sorted_account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    account_transfers = {}
    
    # loop current vault's account_transfers
    for index, (account, transfer) in enumerate(sorted_account_transfers):
        # current vault's single account_transfers stat
        sub_dict = {
            Stats.count: transfer.count,
            Stats.shares: transfer.shares,
            Stats.shares_usdc: float(Decimal(transfer.shares) * share_price)
        }
        account_transfers[account] = sub_dict

        # print top accounts' stats for current vault
        if index < TOP_COUNT:
            print(f"{index}: {account} {sub_dict}")

        # add overall_account_transfers to deposits stats
        overall_account_transfers = deposits_stats[Stats.overall_account_transfers]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + sub_dict[k] for k in sub_dict}

    # add vault's stats to deposits stats
    deposits_stats[Stats.vaults][vault.address] = {**_dict, Stats.account_transfers: account_transfers}
    print()

vault: 0x790a60024bC3aea28385b60480f15a0771f26D09 {count: 27, shares: 204.78834109192803, shares_usdc: 919228.3683873124}
0: 0x4ac68d9a53f699439dc60ddcab304a9d138d7946 {count: 1, shares: 133.84776873416791, shares_usdc: 600799.1734771718}
1: 0xfeb4acf3df3cdea7399794d0869ef76a6efaff52 {count: 1, shares: 29.55273456741059, shares_usdc: 132652.6297001921}
2: 0x7338afb07db145220849b04a45243956f20b14d9 {count: 2, shares: 20.20415112596147, shares_usdc: 90689.87411657044}
3: 0xc4b74ae89ef93e683fb4afba8e118d730313d45a {count: 2, shares: 5.137756217892914, shares_usdc: 23061.719432677342}
4: 0x94e61aea6ad9f699c9c7572b1a2e62661fed98b6 {count: 1, shares: 4.896800101843425, shares_usdc: 21980.145666182067}
5: 0x16388463d60ffe0661cf7f1f31a7d658ac790ff7 {count: 1, shares: 2.6091469776181135, shares_usdc: 11711.61359250408}
6: 0x822f791de0c49f7b73810bf718772b13a69fa9e1 {count: 2, shares: 1.7226825500053868, shares_usdc: 7732.562611950195}
7: 0x2d0904f0f0f0c558975bb142f0d2a1c2d7cc4b5f {count: 1, shar

2022-06-17 09:15:17.810 WARNING connectionpool: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.etherscan.io', port=443): Read timed out. (read timeout=10)")': /api?&apiKey=5RW3TIB9G6X4Y9NP136F1PHJNFP76XNH63&address=0x83d95e0D5f402511dB06817Aff3f9eA88224B030&module=contract&action=getabi
2022-06-17 09:15:32.659 WARNING connectionpool: Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.etherscan.io', port=443): Read timed out. (read timeout=10)")': /api?&apiKey=5RW3TIB9G6X4Y9NP136F1PHJNFP76XNH63&address=0x83d95e0D5f402511dB06817Aff3f9eA88224B030&module=contract&action=getabi
2022-06-17 09:15:51.474 WARNING connectionpool: Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.ether

vault: 0x2a38B9B0201Ca39B17B460eD2f11e4929559071E {count: 8, shares: 1064612.8596445504, shares_usdc: 1093815.1903846003}
0: 0x2955278abce187315d6d72b0d626f1217786df60 {count: 1, shares: 475239.5225149311, shares_usdc: 488275.34261751565}
1: 0x8862720f82884e6466ae8b111d23f6600c55f01f {count: 1, shares: 238045.15891348198, shares_usdc: 244574.7376224788}
2: 0x92be6adb6a12da0ca607f9d87db2f9978cd6ec3e {count: 1, shares: 183167.90850774894, shares_usdc: 188192.2042381165}
3: 0x5e176c4698466243ec90b4ebfde7339ec17c01f3 {count: 1, shares: 66237.220338233, shares_usdc: 68054.10729211074}
4: 0x14a95516ec0e4f54d0935425b12ff5406710bb24 {count: 1, shares: 56076.76813525663, shares_usdc: 57614.95388520672}
5: 0xbf321c8ece1fc1060a653ef66201aef7672960ef {count: 1, shares: 45783.58922959726, shares_usdc: 47039.43308216511}
6: 0x8e52522e6a77578904ddd7f528a22521dc4154f5 {count: 2, shares: 62.69200530121687, shares_usdc: 64.41164700662925}

vault: 0x5c0A86A32c129538D62C106Eb8115a8b02358d57 {count: 405, s

In [10]:
"""top deposits for vaults"""
df = to_reversed_sorted_df(deposits_stats[Stats.vaults])
print(f"top deposits for {Stats.vaults} by approx. USDC value")
df.head(TOP_COUNT)

top deposits for Stats.vaults by approx. USDC value


address,count,shares,approx. USDC value
0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE,2773,4.435867e+08,4.428774e+08
0xdA816459F1AB5631232FE5e97a05BBBb94970c95,1447,3.061457e+08,3.063309e+08
0xa258C4606Ca8206D8aA700cE2143D7db854D168c,3970,1.844362e+05,1.981897e+08
0x27b7b1ad7288079A66d12350c828D3C00A6F07d7,188,1.731556e+08,1.802513e+08
0xdCD90C7f6324cfa40d7169ef80b12031770B4325,1336,1.296601e+05,1.452874e+08
0xB4AdA607B9d6b2c9Ee07A275e9616B84AC560139,110,8.031713e+07,8.101950e+07
0x7Da96a3891Add058AdA2E826306D812C638D87a7,885,6.876278e+07,6.882466e+07
0x5faF6a2D186448Dfa667c51CB3D695c7A6E52d8E,252,6.266090e+04,6.760863e+07
0x67e019bfbd5a67207755D04467D6A70c0B75bF60,294,4.483668e+07,4.777564e+07
0x2DfB14E32e2F8156ec15a2c21c3A6c053af52Be8,82,4.670257e+07,4.704299e+07


In [11]:
"""top deposits for overall_account_transfers"""
df = to_reversed_sorted_df(deposits_stats[Stats.overall_account_transfers])
print(f"top deposits for {Stats.overall_account_transfers.value.replace('_', ' ')} by approx. USDC value")
df.head(TOP_COUNT)

top deposits for overall account transfers by approx. USDC value


address,count,shares,approx. USDC value
0x92be6adb6a12da0ca607f9d87db2f9978cd6ec3e,2840,8.403113e+07,1.544747e+08
0xcc323557c71c0d1d20a1861dc69c06c5f3cc9624,24,1.545123e+08,1.542653e+08
0x6fe02be0ec79dcf582cbdb936d7037d2eb17f661,48,9.170115e+07,9.175663e+07
0x36822d0b11f4594380181ce6e76bd4986d46c389,19,9.159970e+07,9.145324e+07
0x0c8f62939aeee6376f5fac88f48a5a3f2cf5debb,29,8.461024e+07,8.807749e+07
0xb039ea6153c827e59b620bdcd974f7bbfe68214a,14,7.884207e+07,7.888977e+07
0x8e52522e6a77578904ddd7f528a22521dc4154f5,2029,5.248209e+06,6.220596e+07
0xcde35b62c27d70b279cf7d0aa1212ffa9e938cef,1,4.088267e+04,4.411078e+07
0xd96f48665a1410c0cd669a88898eca36b9fc2cce,155,4.314900e+07,4.333270e+07
0xdb91f52eefe537e5256b8043e5f7c7f44d81f5aa,8,3.069951e+07,3.980320e+07


In [12]:
"""helper functions to calculate net transfers"""

def equalize_lists(*args: list[tuple]) -> None:
    """Make all lists equal in size"""
    items = [*args]
    max_length = 0
    for item in items:
        max_length = max(max_length, len(item))
    for item in items:
        item += [(None, None)] * (max_length - len(item))

def get_net_transfer(withdrawals_items: list[tuple[str, dict]], deposits_items: list[tuple[str, dict]]) -> dict[str, dict[str, float]]:
    """Get net transfer of vaults and overall account transfers"""
    net_transfer = {}

    for (withdraw_address, withdraw_stats), (deposit_address, deposit_stats) in zip(withdrawals_items, deposits_items):
        if withdraw_address is not None:
            if net_transfer.get(withdraw_address) is None:
                net_transfer[withdraw_address] = {}
            _dict = {}
            for k in metrics:
                if k == Stats.count:
                    _dict[k] = net_transfer[withdraw_address].get(k, 0) + withdraw_stats[k]
                else:
                    _dict[k] = net_transfer[withdraw_address].get(k, 0) - withdraw_stats[k]
            net_transfer[withdraw_address] = _dict
        if deposit_address is not None:
            if net_transfer.get(deposit_address) is None:
                net_transfer[deposit_address] = {}
            net_transfer[deposit_address] = {k: net_transfer[deposit_address].get(k, 0) + deposit_stats[k] for k in metrics}

    return net_transfer

In [13]:
"""get net transfer of each vault's item in metrics"""
withdrawals_vaults = list(withdrawals_stats[Stats.vaults].items())
deposits_vaults = list(deposits_stats[Stats.vaults].items())
equalize_lists(withdrawals_vaults, deposits_vaults)
net_vaults = get_net_transfer(withdrawals_vaults, deposits_vaults)

"""top net transfer for vaults"""
df = to_reversed_sorted_df(net_vaults)
print(f"top net transfer for vaults by approx. USDC value")
df.head(TOP_COUNT)

top net transfer for vaults by approx. USDC value


address,count,shares,approx. USDC value
0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE,3680,2.207101e+08,2.205130e+08
0x595a68a8c9D5C230001848B69b1947ee2A607164,125,1.193290e+07,1.473966e+07
0xF59D66c1d593Fb10e2f8c2a6fD2C958792434B9c,17,1.103217e+07,1.131669e+07
0x5faF6a2D186448Dfa667c51CB3D695c7A6E52d8E,347,5.953186e+03,6.746548e+06
0xd88dBBA3f9c4391Ee46f5FF548f289054db6E51C,148,2.190488e+06,2.252069e+06
0x5c0A86A32c129538D62C106Eb8115a8b02358d57,610,1.220789e+03,1.333034e+06
0x5AB64C599FcC59f0f2726A300b03166A395578Da,27,1.216129e+06,1.285622e+06
0x2D5D4869381C4Fce34789BC1D38aCCe747E295AE,28,9.881192e+05,9.877763e+05
0x132d8D2C76Db3812403431fAcB00F3453Fc42125,30,8.089265e+02,9.087052e+05
0x6A5468752f8DB94134B6508dAbAC54D3b45efCE6,199,9.312839e+03,5.045091e+05


In [14]:
"""get net transfer of each account's item in metrics scoped by all vaults"""
withdrawals_account_transfers = list(withdrawals_stats[Stats.overall_account_transfers].items())
deposits_account_transfers = list(deposits_stats[Stats.overall_account_transfers].items())
equalize_lists(withdrawals_account_transfers, deposits_account_transfers)
net_overall_account_transfers = get_net_transfer(withdrawals_account_transfers, deposits_account_transfers)

"""top net transfer for overall_account_transfers"""
df = to_reversed_sorted_df(net_overall_account_transfers)
print(f"top net transfer for overall account transfers by approx. USDC value")
df.head(TOP_COUNT)

top net transfer for overall account transfers by approx. USDC value


address,count,shares,approx. USDC value
0x92be6adb6a12da0ca607f9d87db2f9978cd6ec3e,2840,8.403113e+07,1.544747e+08
0xcc323557c71c0d1d20a1861dc69c06c5f3cc9624,24,1.545123e+08,1.542653e+08
0x36822d0b11f4594380181ce6e76bd4986d46c389,19,9.159970e+07,9.145324e+07
0x8e52522e6a77578904ddd7f528a22521dc4154f5,2029,5.248209e+06,6.220596e+07
0xd96f48665a1410c0cd669a88898eca36b9fc2cce,155,4.314900e+07,4.333270e+07
0xa0f75491720835b36edc92d06ddc468d201e9b73,4,2.565635e+07,2.809650e+07
0xf5bce5077908a1b7370b9ae04adc565ebd643966,95,2.457683e+07,2.668887e+07
0xb3295e739380bd68de96802f7c4dba4e54477206,127,1.641976e+04,1.839876e+07
0x5c6374a2ac4ebc38dea0fc1f8716e5ea1add94dd,368,1.812083e+07,1.811750e+07
0x62d9855c399fde8226840ea12d9f1dd693a49b6a,1017,1.787526e+07,1.784668e+07


In [15]:
"""get net transfer of each vault's account's item in metrics scoped by each vault"""
net_transfer = {}
for (withdraw_address, withdraw_stats), (deposit_address, deposit_stats) in zip(withdrawals_vaults, deposits_vaults):
    if withdraw_address is not None:
        if net_transfer.get(withdraw_address) is None:
            net_transfer[withdraw_address] = {}

        for (account, account_transfer) in list(withdraw_stats[Stats.account_transfers].items()):
            if net_transfer[withdraw_address].get(account) is None:
                net_transfer[withdraw_address][account] = {}
            _dict = {}
            for k in metrics:
                if k == Stats.count:
                    _dict[k] = net_transfer[withdraw_address][account].get(k, 0) + account_transfer[k]
                else:
                    _dict[k] = net_transfer[withdraw_address][account].get(k, 0) - account_transfer[k]
            net_transfer[withdraw_address][account] = _dict
        
    if deposit_address is not None:
        if net_transfer.get(deposit_address) is None:
            net_transfer[deposit_address] = {}

        for (account, account_transfer) in list(deposit_stats[Stats.account_transfers].items()):
            if net_transfer[deposit_address].get(account) is None:
                net_transfer[deposit_address][account] = {}
            account_val = {k: net_transfer[deposit_address][account].get(k, 0) + account_transfer[k] for k in metrics}
            net_transfer[deposit_address][account] = account_val

In [17]:
"""top net transfer for each vault's accounts"""
for vault_address, account_transfers in net_transfer.items():
    df = to_reversed_sorted_df(account_transfers)
    print(f"top net transfer for accounts in vault {vault_address} by approx. USDC value")
    print(df.head(TOP_COUNT))
    print()

top net transfer for accounts in vault 0x790a60024bC3aea28385b60480f15a0771f26D09 by approx. USDC value
address                                     count     shares  approx. USDC value
0xfeb4acf3df3cdea7399794d0869ef76a6efaff52      1  29.552735       132652.629700
0x94e61aea6ad9f699c9c7572b1a2e62661fed98b6      1   4.896800        21980.145666
0x16388463d60ffe0661cf7f1f31a7d658ac790ff7      1   2.609147        11711.613593
0x2d0904f0f0f0c558975bb142f0d2a1c2d7cc4b5f      1   1.517349         6810.887533
0x822f791de0c49f7b73810bf718772b13a69fa9e1      3   1.106165         4976.571151
0x0cec743b8ce4ef8802cac0e5df18a180ed8402a7      1   0.913201         4099.061970
0x8c95638ce53c630f600667b5c21fba12885ba042      1   0.900456         4041.854894
0x4ac68d9a53f699439dc60ddcab304a9d138d7946      2   0.000000         2465.124951
0x617f8b1e9818e584689a744710659eff92bb59ce      1   0.495992         2226.349029
0x59a3d47c11c2e1aac52b4720e924d068ef3dfe8a      1   0.474877         2131.569617

top 